# Meteomatics API to SQLite

In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd

from sqlalchemy import create_engine
import sqlite3

## Connect to the Meteomatics Weather API

In [2]:
USERNAME = "hua_kneknas_dimitris"
PASSWORD = "1HlJgKC75q"

LOCATIONS = {
    "Athens": (37.9838, 23.7275),
    "London": (51.5074, -0.1278),
    "Berlin": (52.52, 13.4050),
}

PARAMETERS = (
    "t_2m:C,"                      # Instantaneous air temperature at 2m above ground (°C)
    "t_max_2m_24h:C,"              # Maximum temperature at 2m in the last 24h (°C)
    "t_min_2m_24h:C,"              # Minimum temperature at 2m in the last 24h (°C)
    "relative_humidity_2m:p,"      # Relative humidity at 2m above ground (%)
    "msl_pressure:hPa,"            # Mean sea level atmospheric pressure (hPa)
    "wind_speed_10m:ms,"           # Instantaneous wind speed at 10m above ground (m/s)
    "wind_dir_10m:d,"              # Wind direction at 10m (degrees from north)
    "wind_gusts_10m_24h:ms,"       # Max wind gusts at 10m in the last 24h (m/s)
    "precip_24h:mm"                # Total precipitation in the last 24h (mm)
)

In [3]:
start_date = datetime.utcnow().date()
print(start_date)
end_date = start_date + timedelta(days=6)
print(end_date)

2025-04-10
2025-04-16


In [4]:
time_range = f"{start_date}T00:00:00Z--{end_date}T23:00:00Z:PT1H"
time_range

'2025-04-10T00:00:00Z--2025-04-16T23:00:00Z:PT1H'

In [5]:
def fetch_forecast(city, lat, lon):
    url = f"https://api.meteomatics.com/{time_range}/{PARAMETERS}/{lat},{lon}/json"
    response = requests.get(url, auth=(USERNAME, PASSWORD))

    if response.status_code != 200:
        print(f"Failed for {city}: {response.status_code} {response.text}")
        return []

    data = response.json()
    records = []
    for param in data.get("data", []):
        parameter = param["parameter"]
        for coord in param["coordinates"]:
            for entry in coord["dates"]:
                records.append({
                    "City": city,
                    "Date": entry["date"],
                    parameter: entry["value"]
                })
    return records

In [6]:
# Combine data from all cities
all_records = []
for city, (lat, lon) in LOCATIONS.items():
    records = fetch_forecast(city, lat, lon)
    all_records.extend(records)

# Convert to DataFrame
df = pd.DataFrame(all_records)

# Pivot so each parameter is a column
df_final = df.groupby(['City', 'Date']).first().reset_index()
df_final

,City,Date,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,relative_humidity_2m:p,msl_pressure:hPa,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_24h:ms,precip_24h:mm
0,Athens,2025-04-10T00:00:00Z,5.7,12.7,5.4,85.7,1022.0,0.6,24.9,3.7,0.00
1,Athens,2025-04-10T01:00:00Z,5.3,12.8,4.9,89.0,1022.0,0.8,23.8,3.7,0.00
2,Athens,2025-04-10T02:00:00Z,5.5,12.8,4.9,84.2,1021.0,0.8,13.8,3.7,0.00
3,Athens,2025-04-10T03:00:00Z,5.5,12.8,4.9,81.8,1021.0,0.8,18.2,3.7,0.00
4,Athens,2025-04-10T04:00:00Z,5.7,12.8,4.9,80.9,1021.0,0.6,13.3,3.7,0.00
...,...,...,...,...,...,...,...,...,...,...,...
499,London,2025-04-16T19:00:00Z,11.6,13.9,6.8,48.0,1008.0,3.6,248.0,15.2,0.78
500,London,2025-04-16T20:00:00Z,10.8,13.9,6.8,52.4,1009.0,3.1,251.4,15.2,0.39
501,London,2025-04-16T21:00:00Z,10.0,13.9,6.8,57.2,1010.0,2.7,256.2,15.2,0.00
502,London,2025-04-16T22:00:00Z,9.2,13.9,6.8,62.4,1010.0,2.3,263.0,15.2,0.00


In [7]:
df_final.rename(columns={
    "t_2m:C": "Temperature",
    "t_max_2m_24h:C": "MaxTemp",
    "t_min_2m_24h:C": "MinTemp",
    "relative_humidity_2m:p": "Humidity",
    "msl_pressure:hPa": "Pressure",
    "wind_speed_10m:ms": "WindSpeed",
    "wind_dir_10m:d": "WindDirection",
    "wind_gusts_10m_24h:ms": "WindGusts",
    "precip_24h:mm": "Precipitation"
}, inplace=True)

df_final

,City,Date,Temperature,MaxTemp,MinTemp,Humidity,Pressure,WindSpeed,WindDirection,WindGusts,Precipitation
0,Athens,2025-04-10T00:00:00Z,5.7,12.7,5.4,85.7,1022.0,0.6,24.9,3.7,0.00
1,Athens,2025-04-10T01:00:00Z,5.3,12.8,4.9,89.0,1022.0,0.8,23.8,3.7,0.00
2,Athens,2025-04-10T02:00:00Z,5.5,12.8,4.9,84.2,1021.0,0.8,13.8,3.7,0.00
3,Athens,2025-04-10T03:00:00Z,5.5,12.8,4.9,81.8,1021.0,0.8,18.2,3.7,0.00
4,Athens,2025-04-10T04:00:00Z,5.7,12.8,4.9,80.9,1021.0,0.6,13.3,3.7,0.00
...,...,...,...,...,...,...,...,...,...,...,...
499,London,2025-04-16T19:00:00Z,11.6,13.9,6.8,48.0,1008.0,3.6,248.0,15.2,0.78
500,London,2025-04-16T20:00:00Z,10.8,13.9,6.8,52.4,1009.0,3.1,251.4,15.2,0.39
501,London,2025-04-16T21:00:00Z,10.0,13.9,6.8,57.2,1010.0,2.7,256.2,15.2,0.00
502,London,2025-04-16T22:00:00Z,9.2,13.9,6.8,62.4,1010.0,2.3,263.0,15.2,0.00


In [8]:
df_final['Precipitation'].value_counts()

,count
Precipitation,
0.00,325
0.30,22
6.49,19
2.97,19
0.71,18
...,...
2.37,1
1.77,1
1.17,1


In [9]:
# Save to CSV
output_path = "weather_forecast_all_cities.csv"
df_final.to_csv(output_path, index=False)
print(f"Exported to {output_path}")

Exported to weather_forecast_all_cities.csv


## Store Forecast Data in SQLite Database

In [10]:
# Create SQLite engine (creates a file-based DB named weather.db)
engine = create_engine('sqlite:///weather.db')

# Store the DataFrame into a table called 'weather_forecast'
df_final.to_sql('weather_forecast', con=engine, if_exists='replace', index=False)

print("Weather data stored in weather.db (table: weather_forecast)")

Weather data stored in weather.db (table: weather_forecast)


In [11]:
conn = sqlite3.connect("weather.db")
df = pd.read_sql_query("SELECT * FROM weather_forecast", conn)

conn.close()
df

,City,Date,Temperature,MaxTemp,MinTemp,Humidity,Pressure,WindSpeed,WindDirection,WindGusts,Precipitation
0,Athens,2025-04-10T00:00:00Z,5.7,12.7,5.4,85.7,1022.0,0.6,24.9,3.7,0.00
1,Athens,2025-04-10T01:00:00Z,5.3,12.8,4.9,89.0,1022.0,0.8,23.8,3.7,0.00
2,Athens,2025-04-10T02:00:00Z,5.5,12.8,4.9,84.2,1021.0,0.8,13.8,3.7,0.00
3,Athens,2025-04-10T03:00:00Z,5.5,12.8,4.9,81.8,1021.0,0.8,18.2,3.7,0.00
4,Athens,2025-04-10T04:00:00Z,5.7,12.8,4.9,80.9,1021.0,0.6,13.3,3.7,0.00
...,...,...,...,...,...,...,...,...,...,...,...
499,London,2025-04-16T19:00:00Z,11.6,13.9,6.8,48.0,1008.0,3.6,248.0,15.2,0.78
500,London,2025-04-16T20:00:00Z,10.8,13.9,6.8,52.4,1009.0,3.1,251.4,15.2,0.39
501,London,2025-04-16T21:00:00Z,10.0,13.9,6.8,57.2,1010.0,2.7,256.2,15.2,0.00
502,London,2025-04-16T22:00:00Z,9.2,13.9,6.8,62.4,1010.0,2.3,263.0,15.2,0.00
